In [32]:
import os
import pandas as pd
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

os.chdir('/Users/zebeelou/PhD/compounds-research/')
from compounds_research.compound import utils
from compounds_research import settings
from compounds_research.compound.fetch_exchange_rates import make_exchange_rate_df

In [37]:
tokens = []
for k in settings.COMPOUND_TOKEN_TRANSLATOR:
    tokens.append(settings.COMPOUND_TOKEN_TRANSLATOR[k])

['eth', 'bat', 'zrx', 'usdc', 'dai', 'sai', 'rep', 'wbtc']

In [50]:
# Return dictionaries for where UIP holds and where it doesn't hold
UIP_holds = {}
UIP_not_holds = {}
for quote_token in tokens:
    remaining_tokens = [i for i in tokens if i != quote_token]
    for base_token in remaining_tokens:
        try:
            df = utils.make_ex_rate_and_interest_rate_df(quote_token=quote_token, 
                                                        base_token=base_token, 
                                                        rate_type='borrow_rates', 
                                                        frequency='D', 
                                                        limit=1000)
        except:
            print('Error making dataframe for ' + str(quote_token) + ' to ' + str(base_token))
            continue
        
        df['i_rate_diff'] = df[quote_token] - df[base_token]
        df['ex_rate_diff'] = df[str(base_token)+ '_' + str(quote_token)].diff().shift(-1)
        df = df.dropna()
        model = OLS(endog=df['ex_rate_diff'], exog=sm.add_constant(df['i_rate_diff']))
        res = model.fit(cov_type='HC1')
        # print(res.summary())
        hypotheses = '(const = 0), (i_rate_diff = 1)'
        f_test = res.f_test(hypotheses)
        if f_test.pvalue >= 0.05:
            UIP_holds[str(base_token)+ '_' + str(quote_token)] = f_test.pvalue
        else:
            UIP_not_holds[str(base_token)+ '_' + str(quote_token)] = f_test.pvalue




In [51]:
UIP_holds
# Generally observe that eth_ currency may follow UIP and wbtc_ currency may follow UIP

{'wbtc_eth': array(0.9558718),
 'eth_bat': array(0.2642726),
 'dai_bat': array(0.09326267),
 'rep_bat': array(0.99914414),
 'wbtc_bat': array(0.8401058),
 'eth_zrx': array(0.60902307),
 'rep_zrx': array(0.43421688),
 'wbtc_zrx': array(0.74053023),
 'eth_usdc': array(0.94762153),
 'wbtc_usdc': array(0.90384715),
 'eth_dai': array(0.34658843),
 'rep_dai': array(0.556139),
 'wbtc_dai': array(0.9521816),
 'eth_sai': array(0.88473201),
 'wbtc_sai': array(0.55053974),
 'eth_rep': array(0.94644723),
 'wbtc_rep': array(0.89540767),
 'zrx_wbtc': array(0.77478855),
 'rep_wbtc': array(0.88617388)}